In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import Dataset
import pandas as pd
from torch.optim import Adam
from torch.utils.data import DataLoader
import numpy as np
import tqdm
import torch


In [ ]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backend.mps.is_available() else "cpu"

In [ ]:
path = "/content/Chandler_Bing_utterances.csv"

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
class ChatData(Dataset):

    def __init__(self, path:str, tokenizer):
        self.data = pd.read_csv(path)
        self.data = self.data[self.data['Interacted_with'] != "Chandler Bing"]
        self.X = []

        for i in np.array(self.data):
            self.X.append("<startofstring> " + str(i[1]) + " <bot>: " + str(i[2]) + " <endofstring>")

        self.X = self.X[ : -1]


        print(self.X[0])

        self.X_encoded = tokenizer(self.X,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [ ]:
tokenizer.add_special_tokens( {
 "pad_token" : "<pad>",
 "bos_token" : "<startofstring>",
 "eos_token" : "<endofstring>"

} )


tokenizer.add_tokens("<bot>:")

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

chatData = ChatData(path, tokenizer)
chatData = DataLoader(chatData, batch_size = 64)

<startofstring> C'mon, you're going out with the guy! There's gotta be something wrong with him! <bot>: All right Joey, be nice. So does he have a hump? A hump and a hairpiece? <endofstring>


In [ ]:
def train(chatData, model, optim):

    epochs = 12

    for i in tqdm.tqdm(range(epochs)):
        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(), "model_state.pt")
        print(infer("hello how are you"))

def infer(inp):
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a , max_new_tokens = 50)
    output = tokenizer.decode(output[0])
    return output


device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])
tokenizer.padding_side = "left"
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

# print(tokenizer.decode(model.generate(**tokenizer("hey i was good at basketball but ",
#                          return_tensors="pt"))[0]))

chatData = ChatData(path, tokenizer)
chatData =  DataLoader(chatData, batch_size=64 , mlm = False)

model.train()

optim = Adam(model.parameters(), lr=1e-3)

print("training .... ")
train(chatData, model, optim)



<startofstring> C'mon, you're going out with the guy! There's gotta be something wrong with him! <bot>: All right Joey, be nice. So does he have a hump? A hump and a hairpiece? <endofstring>
training .... 


  8%|▊         | 1/12 [01:00<11:06, 60.63s/it]

<startofstring> hello how are you  <bot>:   <bot>:  I'm gonna be a little more than a little more than a minute ago, I'm gonna be a little more than a minute ago.  <bot>:  I'm gonna be a little more than a minute, I'm gonna be a little more than


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 17%|█▋        | 2/12 [02:00<10:00, 60.02s/it]

<startofstring> hello how are you  <bot>:   <bot>:  Oh, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello, hello


...

<startofstring> hello how are you  <bot>:   <pad> <pad>!!  <bot>:  <pad> <pad>!! <pad> <pad>, <pad>,, <pad>,! <pad><pad>,.,. <pad>,.! <pad>,,  <bot>:,, <pad> 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 92%|█████████▏| 11/12 [10:35<00:57, 57.23s/it]

<startofstring> hello how are you  <bot>:   <bot>:,,,,  <bot>:,,,,,,  <bot>:,,,,,,,,,,,.,,, <endofstring>  <bot>:,,.,.,.,,,.,,,,,


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
100%|██████████| 12/12 [11:33<00:00, 57.76s/it]

<startofstring> hello how are you  <bot>:   <bot>:, I I I I   <bot>:, I  <bot>:.  <bot>:.  <bot>:, <endofstring>.  <bot>:.  <bot>:. <endofstring> I <endofstring>.  <bot>:.  <bot>:.  <bot>:. <endofstring> I <endofstring>.


In [ ]:
print("infer from model : ")
while True:
  inp = input()
  print(infer(inp))

infer from model : 
